# Protein MD Predictor (v1) – Run on Google Colab

This notebook launches the Streamlit app inside Colab and exposes it with a temporary public URL.

**Notes**
- Colab sessions are temporary. If the runtime resets, rerun the cells.
- The public URL is reachable by anyone who has the link. Avoid sensitive data.
- For very large trajectories, Colab may be slow or memory-limited.


## 1) Get the code

Option A (recommended): clone from GitHub.

If you uploaded a ZIP instead, unzip it and `cd` into the project folder.


In [ ]:
!git clone https://github.com/sciencemaths-collab/protein-md-predictor.git
%cd protein-md-predictor


## 2) Install dependencies


In [ ]:
!pip -q install -r requirements.txt


## 3) Download Cloudflared (for a public URL)

Cloudflared creates a temporary `trycloudflare.com` URL without requiring accounts or tokens.


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


## 4) Launch Streamlit + Tunnel

Run this cell, then open the printed **Public URL**.


In [ ]:
import subprocess, time, re

# Start Streamlit in the background
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
time.sleep(3)

# Start cloudflared tunnel and capture the public URL
tunnel_proc = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8501", "--no-autoupdate"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

public_url = None
for _ in range(120):  # up to ~2 minutes
    line = tunnel_proc.stdout.readline()
    if not line:
        time.sleep(1)
        continue
    m = re.search(r"https://[\w-]+\.trycloudflare\.com", line)
    if m:
        public_url = m.group(0)
        break

print("\n✅ Streamlit is running.")
print("🌐 Public URL:", public_url if public_url else "(not found yet; scroll output above)")


## 5) Stop the app (optional)
If you want to stop everything, run the cell below.


In [ ]:
import os, signal
for p in [globals().get('streamlit_proc'), globals().get('tunnel_proc')]:
    if p and p.poll() is None:
        p.send_signal(signal.SIGTERM)
print('Stopped (if running).')
